# Collab Filtering - Recommender anwenden 📈

### Jetzt wird Recommendet! 💯

Aber Fangen wir erst mal wieder mit den Imports an:

In [100]:
import psycopg2
import pandas as pd
from getpass import getpass

### Datenbankverbindung 👨‍💻

In [101]:
user = input("Bitte User für DB eingeben")
pswd = getpass("Bitte User für DB eingeben")
conn = psycopg2.connect(f"dbname='orent001_spotify_test' user='{user}' host='localhost' port='9001' password='{pswd}'")

### Recommendation Table laden 💾

Als erstes lade ich mir die durch ```Collaborative-Filtering_Matrix-Creator``` erstellte Recommendation Tabelle aus dem gespeicherten CSV wieder rein.

In [102]:
similarity_df = pd.read_csv('item_similarity.csv', index_col = 0)
similarity_df

,spotify:track:000xQL6tZNLJzIrtIgxqSl,spotify:track:00BuKLSAFkaEkaVAgIMbeA,spotify:track:00LfFm08VWeZwB0Zlm24AT,spotify:track:00fNdIFKoMxxt8Hnm2kAKL,spotify:track:00lNx0OcTJrS3MKHcB80HY,spotify:track:00qOE7OjRl0BpYiCiweZB2,spotify:track:00xR9dHhuaNznqB4FSzOlr,spotify:track:015IsLQFXbEm0f541N2qoX,spotify:track:01A7PEPSnmtixFPfB2UTal,spotify:track:01DidSmPasiXdPhDVuaULL,...,spotify:track:7zBPzAjKAqQpcv8F8GCq5s,spotify:track:7zBQRGpYImAdIZc97FNj3V,spotify:track:7zFXmv6vqI4qOt4yGf3jYZ,spotify:track:7zNM46fo01dCBidY4yGNTZ,spotify:track:7zTx8ePYAmPFQuxP3xlXZn,spotify:track:7zVCrzzEJU7u24sbJPXA5W,spotify:track:7zWj09xkFgA9tcV6YhfU6q,spotify:track:7zkLpY72g6lKQbiHDqri1S,spotify:track:7zsw78LtXUD7JfEwH64HK2,spotify:track:7zxRMhXxJMQCeDDg0rKAVo
spotify:track:000xQL6tZNLJzIrtIgxqSl,1.000000,-0.005607,0.021245,-0.003562,0.109532,-0.005130,-0.002907,-0.005046,-0.003562,-0.002907,...,-0.007795,-0.002907,0.027908,-0.004604,0.041288,-0.005607,-0.005046,-0.003904,-0.003562,0.031641
spotify:track:00BuKLSAFkaEkaVAgIMbeA,-0.005607,1.000000,0.033348,0.037933,-0.004736,-0.006820,-0.003865,0.023509,-0.004736,-0.003865,...,-0.010363,-0.003865,-0.007947,-0.006121,-0.005870,-0.007455,0.053727,-0.005190,0.037933,-0.007249
spotify:track:00LfFm08VWeZwB0Zlm24AT,0.021245,0.033348,1.000000,0.027540,-0.006045,-0.008705,-0.004934,0.015222,-0.006045,-0.004934,...,-0.013227,-0.004934,0.050234,-0.007812,0.019653,-0.009516,0.039006,-0.006624,-0.006045,0.012778
spotify:track:00fNdIFKoMxxt8Hnm2kAKL,-0.003562,0.037933,0.027540,1.000000,0.063858,-0.004333,-0.002456,0.090445,-0.003009,-0.002456,...,-0.006584,-0.002456,-0.005049,-0.003889,-0.003729,-0.004736,0.043092,-0.003297,-0.003009,-0.004606
spotify:track:00lNx0OcTJrS3MKHcB80HY,0.109532,-0.004736,-0.006045,0.063858,1.000000,-0.004333,-0.002456,-0.004262,-0.003009,-0.002456,...,0.024325,-0.002456,0.035021,-0.003889,-0.003729,-0.004736,-0.004262,-0.003297,-0.003009,-0.004606
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
spotify:track:7zVCrzzEJU7u24sbJPXA5W,-0.005607,-0.007455,-0.009516,-0.004736,-0.004736,-0.006820,-0.003865,-0.006708,-0.004736,-0.003865,...,-0.010363,-0.003865,-0.007947,0.026964,-0.005870,1.000000,0.023509,-0.005190,-0.004736,-0.007249
spotify:track:7zWj09xkFgA9tcV6YhfU6q,-0.005046,0.053727,0.039006,0.043092,-0.004262,-0.006137,-0.003478,0.061033,-0.004262,0.054489,...,-0.009325,-0.003478,-0.007151,0.031209,-0.005282,0.023509,1.000000,-0.004670,-0.004262,0.024539
spotify:track:7zkLpY72g6lKQbiHDqri1S,-0.003904,-0.005190,-0.006624,-0.003297,-0.003297,0.037794,-0.002691,-0.004670,-0.003297,-0.002691,...,-0.007214,-0.002691,-0.005532,-0.004261,-0.004086,-0.005190,-0.004670,1.000000,-0.003297,-0.005047
spotify:track:7zsw78LtXUD7JfEwH64HK2,-0.003562,0.037933,-0.006045,-0.003009,-0.003009,-0.004333,-0.002456,-0.004262,-0.003009,-0.002456,...,-0.006584,-0.002456,-0.005049,-0.003889,-0.003729,-0.004736,-0.004262,-0.003297,1.000000,-0.004606


## Songs raussuchen, die in einer Playlist sein könnten ☃️❄️🎄🎅🏼

Um meinen Recommender zu prüfen, habe ich mir Song URIs rausgesucht von einem Lied mit 'Christ' und mit 'Snow' im Namen.

Die Wahl fiel auf die Lieder ```Let It Snow! Let It Snow! Let It Snow!``` und ```It's Beginning To Look A Lot Like Christmas```, eindeutig zwei Weihnachtslieder.

#### ⚠️ Wichtig!
Es gilt zu beachten, dass in der ```Collaborative-Filtering_Matrix-Creator``` viele Songs, die in zu wenigen Playlists sind, rausgeschmissen wurden (aktuell, vielleicht ändere ich es ja wieder). Das heißt, nicht jeder Song, den man sich hiermit raussucht, um sich Recommendations vorschlagen zu lassen, wird funktionieren. Vielleicht sollte man noch Abfangen von Fehlern mit einbauen, wenn es die Song URI nicht im Recommendation Datensatz gibt.

In [118]:
query = f"SELECT track_name, song_id FROM Song where track_name LIKE '%Christ%';"
pd.io.sql.read_sql_query(query,conn)

,track_name,song_id
0,It's Beginning To Look A Lot Like Christmas,spotify:track:6yaxdh87KVj82QIYKTP1zt
1,White Christmas,spotify:track:4so0Wek9Ig1p6CRCHuINwW
2,The Christmas Song (Merry Christmas To You),spotify:track:5humEOJehpAwMJEJmb0JJ2
3,Last Christmas,spotify:track:2FRnf9qhLbvw8fu4IBXx78
4,Have Yourself A Merry Little Christmas - 1999 ...,spotify:track:2FPfeYlrbSBR8PwCU0zaqq
...,...,...
967,"Christ, Or Else I Die",spotify:track:0JTF7biP90iGcgAtE0gFbs
968,All I Have Is Christ,spotify:track:4yS3Dl3yLXoZgeXwZbKHra
969,All Alone On Christmas,spotify:track:7e8F3GrWIgoqI92iBqVYrJ
970,Christmas Time Is Here,spotify:track:47Jo9CnbSscAlp0LyoooPu


## Das Recommendation Herz ❤️

```get_similar_songs``` generiert das Rating für Songs zu einem Song 🎵

In [128]:
def get_similar_songs(songURL):
    similar_score = similarity_df[songURL]
    similar_score = similar_score.sort_values(ascending=False)
    return similar_score

## Recommendation Herz im Playlist Einsatz 💕

```get_similar_playlist``` generiert mit Hilfe von ```get_similar_songs``` das Rating für Songs zu einer Reihe von Songs, also zu Playlists 🎶



In [129]:
def get_similar_playlist(songs):
    similar_songs = pd.DataFrame()
    for song in songs:
        similar_songs = similar_songs.append(get_similar_songs(song), ignore_index=True)
    similar_songs = similar_songs.sum().sort_values(ascending=False).to_frame()
    # Die ersten len(songs) Vorschläge werden die Lieder selbst dein, daher weg damit!
    similar_songs = similar_songs.iloc[len(songs): , :]
    similar_songs.reset_index(level=0, inplace=True)
    similar_songs.columns = ['TrackURL', 'Rating']
    return similar_songs

similar_songs = get_similar_playlist(['spotify:track:7inXu0Eaeg02VsM8kHNvzM', 'spotify:track:6yaxdh87KVj82QIYKTP1zt'])
similar_songs

,TrackURL,Rating
0,spotify:track:4so0Wek9Ig1p6CRCHuINwW,0.957572
1,spotify:track:65Q2mv3UlVi9eO70OpsmSe,0.872007
2,spotify:track:5hslUAKq9I9CG2bAulFkHN,0.868073
3,spotify:track:3T7dNA7O8c3Axj5WyDNcH3,0.866942
4,spotify:track:6UrKVLhzBYZY6fXmNyCLDi,0.817367
...,...,...
5959,spotify:track:3a1lNhkSLSkpJE4MSHpDu9,-0.022394
5960,spotify:track:7KXjTSCq5nL1LoYtL7XAwS,-0.023507
5961,spotify:track:7yyRTcZmCiyzzJlNzGC9Ol,-0.023507
5962,spotify:track:1xznGGDReH1oQq0xzbwXa3,-0.023735


## Welches Lied steckt aber jetzt hinter dieser doofen URL? 🤔

Um unsere Liedvorschläge jetzt noch wenigstens per Augenmaß zu validieren, schauen wir uns mal die richtigen Titel der 5 am besten und am schlechtesten passendenden Lieder zu unseren zwei Weihnachtsliedern an:

In [130]:
def similar_playlist_get_names(similar_song_table):
    song_names = []
    for trackURL in similar_song_table['TrackURL']:
        query = f"SELECT track_name FROM Song WHERE song_id = '{trackURL}';"
        song_names.append(pd.io.sql.read_sql_query(query,conn)['track_name'][0])
    similar_song_table['song_names'] = song_names
    return similar_song_table

similar_songs_with_name = similar_playlist_get_names(similar_songs[0:5].append(similar_songs[-6:-1])) 
similar_songs_with_name

,TrackURL,Rating,song_names
0,spotify:track:4so0Wek9Ig1p6CRCHuINwW,0.957572,White Christmas
1,spotify:track:65Q2mv3UlVi9eO70OpsmSe,0.872007,A Holly Jolly Christmas - Single Version
2,spotify:track:5hslUAKq9I9CG2bAulFkHN,0.868073,It's the Most Wonderful Time of the Year
3,spotify:track:3T7dNA7O8c3Axj5WyDNcH3,0.866942,Rockin' Around The Christmas Tree - Single Ver...
4,spotify:track:6UrKVLhzBYZY6fXmNyCLDi,0.817367,Mele Kalikimaka - Single Version
5958,spotify:track:5hTpBe8h35rJ67eAWHQsJx,-0.021482,Caroline
5959,spotify:track:3a1lNhkSLSkpJE4MSHpDu9,-0.022394,Congratulations
5960,spotify:track:7KXjTSCq5nL1LoYtL7XAwS,-0.023507,HUMBLE.
5961,spotify:track:7yyRTcZmCiyzzJlNzGC9Ol,-0.023507,Broccoli (feat. Lil Yachty)
5962,spotify:track:1xznGGDReH1oQq0xzbwXa3,-0.023735,One Dance


# 🎉 Ja GEIL! 🥳